In [4]:
import requests
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from ast import literal_eval
from tqdm import tqdm

# DATA_SCRAP

In [ ]:
url = "https://api.opendota.com/api/matches/5939912593"
lol = {"http":'172.67.0.9:80', "http":'172.67.180.30:80'}
for i in range (130):
  pass
  match_response = requests.get(url, proxies=lol)
  #print(match_response.json())
print(match_response.json())

{'match_id': 5939912593, 'barracks_status_dire': 63, 'barracks_status_radiant': 3, 'chat': [{'time': -86, 'type': 'chat', 'key': '1 sec', 'slot': 2, 'player_slot': 2}, {'time': -86, 'type': 'chat', 'key': 'lags', 'slot': 2, 'player_slot': 2}, {'time': -86, 'type': 'chatwheel', 'key': '70001', 'slot': 5, 'player_slot': 128}, {'time': -86, 'type': 'chat', 'key': 'g?', 'slot': 1, 'player_slot': 1}, {'time': -86, 'type': 'chatwheel', 'key': '7', 'slot': 4, 'player_slot': 4}, {'time': -86, 'type': 'chatwheel', 'key': '167', 'slot': 4, 'player_slot': 4}, {'time': -86, 'type': 'chatwheel', 'key': '121001', 'slot': 3, 'player_slot': 3}, {'time': -86, 'type': 'chat', 'key': 'g', 'slot': 9, 'player_slot': 132}, {'time': -86, 'type': 'chatwheel', 'key': '121001', 'slot': 3, 'player_slot': 3}, {'time': -86, 'type': 'chatwheel', 'key': '167', 'slot': 4, 'player_slot': 4}, {'time': -85, 'type': 'chatwheel', 'key': '10', 'slot': 3, 'player_slot': 3}, {'time': -82, 'type': 'chat', 'key': 'gl hf', 'slo

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/dota_dataset/picks_bans_2023_01.csv')
df

,Unnamed: 0,is_pick,hero_id,team,order,ord,match_id,leagueid
0,0,False,107.0,0.0,0.0,0.0,5939912593,12903
1,1,False,13.0,1.0,1.0,1.0,5939912593,12903
2,2,False,98.0,0.0,2.0,2.0,5939912593,12903
3,3,False,21.0,1.0,3.0,3.0,5939912593,12903
4,4,True,126.0,0.0,4.0,4.0,5939912593,12903
...,...,...,...,...,...,...,...,...
147317,147317,False,114.0,0.0,19.0,19.0,6996333085,14999
147318,147318,False,93.0,1.0,20.0,20.0,6996333085,14999
147319,147319,False,10.0,0.0,21.0,21.0,6996333085,14999
147320,147320,True,13.0,1.0,22.0,22.0,6996333085,14999


In [ ]:
df.nunique()

Unnamed: 0    147322
is_pick            2
hero_id          123
team               2
order             24
ord               24
match_id        6159
leagueid          36
dtype: int64

In [ ]:
df.drop('ord', 1, inplace=True)
df.drop('leagueid', 1, inplace=True)
df.drop('Unnamed: 0', 1, inplace= True)
df = df[df.is_pick != False]
df.drop('is_pick', 1, inplace=True)
df.drop('order', 1, inplace=True)

<ipython-input-25-5d4b0256cf92>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop('ord', 1, inplace=True)
<ipython-input-25-5d4b0256cf92>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop('leagueid', 1, inplace=True)
<ipython-input-25-5d4b0256cf92>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop('Unnamed: 0', 1, inplace= True)
<ipython-input-25-5d4b0256cf92>:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop('is_pick', 1, inplace=True)
<ipython-input-25-5d4b0256cf92>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas

In [ ]:
df

,hero_id,team,match_id
4,126.0,0.0,5939912593
5,91.0,1.0,5939912593
6,70.0,1.0,5939912593
7,33.0,0.0,5939912593
14,27.0,1.0,5939912593
...,...,...,...
147313,53.0,1.0,6996333085
147314,71.0,1.0,6996333085
147315,129.0,0.0,6996333085
147320,13.0,1.0,6996333085


In [ ]:
def radiant_dire_picks(match_id):
  dummy_df = df[df.match_id == match_id]
  radiant_pick = []
  dire_pick = []
  for ind in dummy_df.index:
    if (dummy_df['team'][ind] == 0):
      radiant_pick.append(dummy_df['hero_id'][ind])
    else:
      dire_pick.append(dummy_df['hero_id'][ind])
  return radiant_pick, dire_pick

In [ ]:
url = "https://api.opendota.com/api/matches"
prox_list = [{"http":'31.7.38.227:8080	'}, {"http":'58.234.116.197:80'}, {"http":'50.217.226.45:80'}, {"http":'47.74.226.8:5001'}, {"http":'45.95.147.107:8080'}, {"http":'172.67.182.111:80'},
             {"http":'45.8.105.83:80'},{"http":'45.8.107.194:80'},{"http":'45.8.104.198:80'},{"http":'185.162.230.151:80'},{"http":'185.162.229.48:80'},{"http":'45.8.106.120:80'}]

def win_parse(match_id):
  new_url = url + '/' + str(match_id)
  while True:
    for proxy in prox_list:
      match_response = requests.get(new_url, proxies = proxy)
      if (match_response.json() == {'error': 'rate limit exceeded'}):
        continue
      return match_response.json()["radiant_win"]
  raise Exception("Bad proxies")

In [ ]:
my_df = pd.DataFrame(columns= ["match_id", "pick_radiant", "pick_dire", "radiant_win"])
#my_df.loc[len(my_df.index)] = ['Amy', 89, 93, 93]
my_df

,match_id,pick_radiant,pick_dire,radiant_win


In [ ]:
ids = df.match_id.unique()
iter = 0
for id in tqdm(ids):
  if (iter < 4613):
    iter += 1
    continue
  result = win_parse(id)
  r, d = radiant_dire_picks(id)
  my_df.loc[len(my_df.index)] = [id, r, d, result]

100%|██████████| 6159/6159 [25:27<00:00,  4.03it/s]


In [ ]:
match_response = requests.get("https://api.opendota.com/api/matches/6972310511", proxies={"http":'172.67.0.9:80'})
print(match_response.json())

{'match_id': 6972310511, 'barracks_status_dire': 63, 'barracks_status_radiant': 63, 'chat': [{'time': -189, 'type': 'chatwheel', 'key': '0', 'slot': 2, 'player_slot': 2}, {'time': -162, 'type': 'chatwheel', 'key': '3', 'slot': 2, 'player_slot': 2}, {'time': -82, 'type': 'chatwheel', 'key': '33001', 'slot': 8, 'player_slot': 131}, {'time': -77, 'type': 'chatwheel', 'key': '165', 'slot': 7, 'player_slot': 130}, {'time': -77, 'type': 'chatwheel', 'key': '165', 'slot': 7, 'player_slot': 130}, {'time': -73, 'type': 'chat', 'key': 'gl hf ', 'slot': 8, 'player_slot': 131}, {'time': -69, 'type': 'chat', 'key': 'glhf', 'slot': 2, 'player_slot': 2}, {'time': -67, 'type': 'chatwheel', 'key': '137001', 'slot': 9, 'player_slot': 132}, {'time': -67, 'type': 'chatwheel', 'key': '137001', 'slot': 9, 'player_slot': 132}, {'time': -67, 'type': 'chat', 'key': 'gl hf', 'slot': 1, 'player_slot': 1}, {'time': -66, 'type': 'chatwheel', 'key': '33001', 'slot': 8, 'player_slot': 131}, {'time': -64, 'type': 'ch

In [ ]:
df.match_id.unique()[4613]

6972310511

In [ ]:
my_df

,match_id,pick_radiant,pick_dire,radiant_win
0,5939912593,"[126.0, 33.0, 100.0, 109.0, 121.0]","[91.0, 70.0, 27.0, 38.0, 11.0]",False
1,5939925225,"[50.0, 51.0, 36.0, 67.0, 126.0]","[83.0, 27.0, 2.0, 94.0, 74.0]",True
2,5940024896,"[121.0, 107.0, 98.0, 54.0, 126.0]","[91.0, 76.0, 74.0, 1.0, 85.0]",True
3,5940069746,"[112.0, 110.0, 17.0, 29.0, 10.0]","[111.0, 36.0, 88.0, 13.0, 93.0]",False
4,5940146563,"[103.0, 41.0, 86.0, 4.0, 106.0]","[64.0, 12.0, 29.0, 26.0, 126.0]",False
...,...,...,...,...
6154,6996220882,"[87.0, 86.0, 43.0, 81.0, 13.0]","[51.0, 58.0, 137.0, 56.0, 106.0]",True
6155,6996269306,"[25.0, 136.0, 43.0, 135.0, 23.0]","[61.0, 86.0, 137.0, 10.0, 62.0]",True
6156,6996300956,"[83.0, 38.0, 123.0, 126.0, 10.0]","[86.0, 137.0, 5.0, 46.0, 19.0]",True
6157,6996310238,"[25.0, 136.0, 51.0, 52.0, 53.0]","[86.0, 96.0, 83.0, 90.0, 67.0]",False


In [ ]:
my_df.to_csv("/content/drive/MyDrive/dota_dataset/picks_wins_2023_01.csv")

In [ ]:
print(my_df["radiant_win"].value_counts()[True])
print(my_df["radiant_win"].value_counts()[False])

2958
3201


# MODEL BUILD

In [18]:
df = pd.read_csv('/content/drive/MyDrive/dota_dataset/picks_wins_2023_01.csv')
df['pick_dire'] = df['pick_dire'].apply(literal_eval)
df['pick_radiant'] = df['pick_radiant'].apply(literal_eval)

In [ ]:
class DotaDataset(Dataset):
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        pick_radiant = torch.tensor(row['pick_radiant'])
        pick_dire = torch.tensor(row['pick_dire'])
        target = torch.tensor(int(row['radiant_win'] == True))
        return pick_radiant, pick_dire, target

In [ ]:
class DotaModel(nn.Module):
  def __init__(self, input_size, hidden1_size, hidden2_size, num_classes):
    super().__init__()
    self.fc1 = nn.Linear(input_size, hidden1_size)
    self.relu1 = nn.ReLU()
    self.fc2 = nn.Linear(hidden1_size, hidden2_size)
    self.relu2 = nn.ReLU()
    self.fc3 = nn.Linear(hidden2_size, hidden2_size)
    self.relu3 = nn.ReLU()
    self.fc4 = nn.Linear(hidden2_size, num_classes)

  def forward(self, x):
    out = self.fc1(x)
    out = self.relu1(out)
    out = self.fc2(out)
    out = self.relu2(out)
    out = self.fc3(out)
    out = self.relu3(out)
    out = self.fc4(out)
    return out

model = DotaModel(10, 130, 130, 1)

In [ ]:
train, test = train_test_split(df, test_size=0.2)

dataset = DotaDataset(train)
criterion = torch.nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001,weight_decay = 0.01)

model.train()
loss_sum = 0
n = 0
epoch_num = 10
for now_epoch in range(epoch_num):
    pbar = tqdm(range(len(dataset)))
    print('--------------epoch {}: ------------------'.format(now_epoch))
    for i in pbar:
        pick_r, pick_d, target = dataset[i]
        x = torch.cat((pick_r, pick_d))
        out = model(x)
        loss = criterion(out, target)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.01)
        optimizer.step()
        loss_sum+=loss.item()
        n+=1
        pbar.set_description(f'running loss: {(loss_sum/n):.4f},current loss: {(loss.item()):.4f}')

running loss: 0.4489,current loss: 0.0030:   0%|          | 14/4927 [00:00<00:35, 136.80it/s]

--------------epoch 0: ------------------


running loss: 0.4819,current loss: 0.0001:   0%|          | 20/4927 [00:00<00:25, 193.49it/s]

--------------epoch 1: ------------------


running loss: 0.4820,current loss: 0.9986:   0%|          | 17/4927 [00:00<00:29, 167.39it/s]

...

running loss: 0.4821,current loss: 0.9993:   0%|          | 19/4927 [00:00<00:26, 184.66it/s]

--------------epoch 8: ------------------


running loss: 0.4821,current loss: 0.9995:   0%|          | 18/4927 [00:00<00:28, 172.62it/s]

--------------epoch 9: ------------------


running loss: 0.4821,current loss: 0.9982: 100%|██████████| 4927/4927 [00:28<00:00, 174.85it/s]


In [ ]:
dataset = DotaDataset(test)
predictions = []
model.eval()

pbar = tqdm(range(len(dataset)))
for i in pbar:
    pick_r, pick_d, target = dataset[i]
    x = torch.cat((pick_r, pick_d))
    out = model(x)
    predictions.append(out.detach().numpy())

def score_tile(predictions, data):
    score = 0
    for i in range(len(dataset)):
        pred = predictions[i][0]
        truth = data.iloc[i]['radiant_win']
        score += abs(pred-truth)
    score /= len(dataset)
    return 1 - score
score_tile(predictions, test)

100%|██████████| 1232/1232 [00:00<00:00, 2354.78it/s]


0.5258746620527674

# Random forest try

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
df0 = pd.DataFrame(columns=['0_radiant','1_radiant','2_radiant','3_radiant','4_radiant','0_dire','1_dire','2_dire','3_dire','4_dire', 'radiant_win'])
for i in range(len(df)):
  df0.loc[len(df0.index)] = [df['pick_radiant'][i][0], df['pick_radiant'][i][1], df['pick_radiant'][i][2], df['pick_radiant'][i][3], df['pick_radiant'][i][4], df['pick_dire'][i][0], df['pick_dire'][i][1], df['pick_dire'][i][2], df['pick_dire'][i][3], df['pick_dire'][i][4], df['radiant_win'][i]]

df0.radiant_win = df0.radiant_win.replace({True: 1, False: 0})
y = df0['radiant_win']
x = df0.drop(['radiant_win'], axis = 1)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [ ]:
rf_model = RandomForestClassifier()
rf_model.fit(x_train, y_train)

RandomForestClassifier()

In [ ]:
rf_model.score(x_test, y_test)

0.5097402597402597

# Cat boost try

In [1]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 9.0 MB/s eta 0:00:00


In [3]:
import numpy as np

from catboost import CatBoostClassifier, Pool


In [19]:
df = df.drop(['Unnamed: 0'], axis = 1)

In [20]:
df0 = pd.DataFrame(columns=['0_radiant','1_radiant','2_radiant','3_radiant','4_radiant','0_dire','1_dire','2_dire','3_dire','4_dire', 'radiant_win'])
for i in range(len(df)):
  df0.loc[len(df0.index)] = [df['pick_radiant'][i][0], df['pick_radiant'][i][1], df['pick_radiant'][i][2], df['pick_radiant'][i][3], df['pick_radiant'][i][4], df['pick_dire'][i][0], df['pick_dire'][i][1], df['pick_dire'][i][2], df['pick_dire'][i][3], df['pick_dire'][i][4], df['radiant_win'][i]]

df0.radiant_win = df0.radiant_win.replace({True: 1, False: 0})
y = df0['radiant_win']
x = df0.drop(['radiant_win'], axis = 1)

In [21]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [22]:
model = CatBoostClassifier(learning_rate=0.001,
                           loss_function='Logloss',
                           verbose=True)

In [23]:
model.fit(x_train, y_train)

0:	learn: 0.6931237	total: 57.8ms	remaining: 57.8s
1:	learn: 0.6931063	total: 64.6ms	remaining: 32.2s
2:	learn: 0.6930895	total: 67.4ms	remaining: 22.4s
3:	learn: 0.6930728	total: 70.5ms	remaining: 17.6s
4:	learn: 0.6930599	total: 73.8ms	remaining: 14.7s
5:	learn: 0.6930434	total: 76.8ms	remaining: 12.7s
6:	learn: 0.6930294	total: 80.1ms	remaining: 11.4s
7:	learn: 0.6930176	total: 83.2ms	remaining: 10.3s
8:	learn: 0.6929940	total: 86ms	remaining: 9.47s
9:	learn: 0.6929750	total: 89ms	remaining: 8.81s
10:	learn: 0.6929568	total: 95.3ms	remaining: 8.57s
11:	learn: 0.6929435	total: 100ms	remaining: 8.23s
12:	learn: 0.6929238	total: 104ms	remaining: 7.9s
13:	learn: 0.6929048	total: 108ms	remaining: 7.63s
14:	learn: 0.6928890	total: 114ms	remaining: 7.49s
15:	learn: 0.6928699	total: 120ms	remaining: 7.38s
16:	learn: 0.6928451	total: 123ms	remaining: 7.13s
17:	learn: 0.6928269	total: 126ms	remaining: 6.88s
18:	learn: 0.6928097	total: 129ms	remaining: 6.66s
19:	learn: 0.6927943	total: 132ms	r

In [24]:
test_data = Pool(x_test, y_test)

In [25]:
preds_class = model.predict(test_data)

In [26]:
preds_proba = model.predict_proba(test_data)

In [35]:
print("class = ", preds_class)
print("proba = ", preds_proba)

class =  [0 0 0 ... 0 0 0]
proba =  [[0.51583451 0.48416549]
 [0.52944131 0.47055869]
 [0.50958969 0.49041031]
 ...
 [0.51670443 0.48329557]
 [0.51277478 0.48722522]
 [0.51892344 0.48107656]]


In [47]:
counter = 0
i = 0
for x in y_test:
  if((x - int(preds_class[i])) == 0):
    counter += 1
  i += 1
print(counter/len(y_test))

0.5170454545454546
